In [1]:
from pyspark.shell import spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 12:19:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.3
      /_/

Using Python version 3.9.6 (default, Nov 11 2024 03:15:38)
Spark context Web UI available at http://localhost:4040
Spark context available as 'sc' (master = local[*], app id = local-1742971799845).
SparkSession available as 'spark'.


In [2]:
spark = SparkSession.builder.appName("SparkTesting").getOrCreate()

25/03/26 12:20:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [23]:
data = [("This is a sentence.",), ("Another sentence here.",), ("Final example.",)]
df = spark.createDataFrame(data, ["sentences"])
df.show(truncate=False)

+----------------------+
|sentences             |
+----------------------+
|This is a sentence.   |
|Another sentence here.|
|Final example.        |
+----------------------+



In [24]:
df_words = df.withColumn("words", F.explode(F.split(F.col("sentences"), " ")))
df_words.show()

+--------------------+---------+
|           sentences|    words|
+--------------------+---------+
| This is a sentence.|     This|
| This is a sentence.|       is|
| This is a sentence.|        a|
| This is a sentence.|sentence.|
|Another sentence ...|  Another|
|Another sentence ...| sentence|
|Another sentence ...|    here.|
|      Final example.|    Final|
|      Final example.| example.|
+--------------------+---------+



In [28]:
data = [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]
df = spark.createDataFrame(data, ["number"])

In [33]:
df.rdd.getNumPartitions()

12

In [34]:
df_coalesced = df.coalesce(6)
df_coalesced.rdd.getNumPartitions()

6

In [37]:
df_increased = df_coalesced.coalesce(10)
df_increased.rdd.getNumPartitions()

10

In [43]:
# from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
import random

# Initialize Spark Session
# spark = SparkSession.builder.appName("LargeDataFrame").getOrCreate()

# Define Schema
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("salary", IntegerType(), False),
    StructField("department", StringType(), False)
])

# Generate 10,000 rows of data
data = [(i, f"Name_{i}", random.randint(18, 65), random.randint(30000, 120000),
         random.choice(["HR", "IT", "Finance", "Marketing"])) for i in range(1, 10001)]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)
df = df.coalesce(1)

# Save DataFrame as Parquet file
parquet_path = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/parquet_files/large_dataset_single.parquet"
df.write.mode("overwrite").parquet(parquet_path)

print(f"Parquet file saved at: {parquet_path}")


Parquet file saved at: /Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/parquet_files/large_dataset_single.parquet


In [45]:
spark.stop()

ConnectionRefusedError: [Errno 61] Connection refused

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# Create Spark session
spark = SparkSession.builder.appName("JoinExample").getOrCreate()

# Create languages DataFrame
data1 = [("1", "Java", "20000"),
         ("2", "Python", "100000"),
         ("3", "Scala", "3000")]

languages = spark.createDataFrame(data1, ["id", "language", "tution_fees"])
languages.createOrReplaceTempView("languages")

# Create students DataFrame
data2 = [("1", "studentA"), ("1", "studentB"),
         ("2", "studentA"), ("3", "studentC")]

students = spark.createDataFrame(data2, ["language_id", "studentName"])
students.createOrReplaceTempView("students")

# Perform join and aggregation using SQL
df = spark.sql("""
    SELECT students.studentName, SUM(CAST(students.language_id AS INT)) as c
    FROM students
    INNER JOIN languages
    ON students.language_id = languages.id
    WHERE students.studentName = 'studentA'
    GROUP BY students.studentName
""")

df.explain(extended=False)

df.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[studentName#36], functions=[sum(cast(language_id#35 as int))])
   +- Exchange hashpartitioning(studentName#36, 200), ENSURE_REQUIREMENTS, [plan_id=273]
      +- HashAggregate(keys=[studentName#36], functions=[partial_sum(cast(language_id#35 as int))])
         +- Project [language_id#35, studentName#36]
            +- SortMergeJoin [language_id#35], [id#29], Inner
               :- Sort [language_id#35 ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(language_id#35, 200), ENSURE_REQUIREMENTS, [plan_id=265]
               :     +- Filter ((isnotnull(studentName#36) AND (studentName#36 = studentA)) AND isnotnull(language_id#35))
               :        +- Scan ExistingRDD[language_id#35,studentName#36]
               +- Sort [id#29 ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(id#29, 200), ENSURE_REQUIREMENTS, [plan_id=266]
                     +- Project [i

In [3]:
df.explain(extended=True)

== Parsed Logical Plan ==
'Aggregate ['students.studentName], ['students.studentName, 'SUM(cast('students.language_id as int)) AS c#39]
+- 'Filter ('students.studentName = studentA)
   +- 'Join Inner, ('students.language_id = 'languages.id)
      :- 'UnresolvedRelation [students], [], false
      +- 'UnresolvedRelation [languages], [], false

== Analyzed Logical Plan ==
studentName: string, c: bigint
Aggregate [studentName#36], [studentName#36, sum(cast(language_id#35 as int)) AS c#39L]
+- Filter (studentName#36 = studentA)
   +- Join Inner, (language_id#35 = id#29)
      :- SubqueryAlias students
      :  +- View (`students`, [language_id#35,studentName#36])
      :     +- LogicalRDD [language_id#35, studentName#36], false
      +- SubqueryAlias languages
         +- View (`languages`, [id#29,language#30,tution_fees#31])
            +- LogicalRDD [id#29, language#30, tution_fees#31], false

== Optimized Logical Plan ==
Aggregate [studentName#36], [studentName#36, sum(cast(language_id#

In [4]:
# Access the JVM's SizeEstimator from PySpark
df_jvm = df._jdf  # Get the Java DataFrame reference
size_estimator = spark._jvm.org.apache.spark.util.SizeEstimator

# Estimate size in bytes
estimated_size_bytes = size_estimator.estimate(df_jvm)

# Convert to MB
estimated_size_mb = estimated_size_bytes / 1_000_000

print(f"Estimated size of the DataFrame = {estimated_size_mb:.2f} MB")

Estimated size of the DataFrame = 4.79 MB


In [ ]:
from pyspark.sql.functions import col, upper, lower, to_date, date_format, regexp_replace, udf, substring
from pyspark.sql.types import FloatType, IntegerType, StringType, DateType
import datetime

def apply_field_conversions(pDF, aAsset, aLogger=None):
    df = pDF

    def log(msg): aLogger.info(msg) if aLogger else print(msg)

    def cast_to(target):
        type_map = {
            "String": StringType(),
            "Int": IntegerType(),
            "Float": FloatType(),
            "Date": DateType()
        }
        return lambda c: c.cast(type_map[target])

    def date_to_days_udf():
        def date_to_days(d):
            if d:
                base_date = datetime.date(1900, 1, 1)
                if isinstance(d, datetime.datetime):
                    d = d.date()
                return (d - base_date).days if isinstance(d, datetime.date) else None
            return None
        return udf(date_to_days, IntegerType())

    def get_handler(rule, source_type, target_type):
        rule = rule.strip()

        if rule.startswith("Left:"):
            n = int(rule.split("Left:")[1])
            return lambda c: substring(c, 1, n)

        if rule == "???":
            rule = "Currency:Dollar"

        if rule.startswith("Date:"):
            fmt = rule.split("Date:")[1]
            if source_type == "String":
                return lambda c: to_date(c, fmt)
            elif source_type == "Date":
                return lambda c: date_format(c, fmt)
            else:
                raise ValueError(f"Invalid source type for rule {rule}: {source_type}")

        handler_map = {
            "Upper": lambda c: upper(c),
            "Lower": lambda c: lower(c),
            "Decimal:Comma": lambda c: regexp_replace(c, ",", ".").cast(FloatType()),
            "Currency:Dollar": lambda c: regexp_replace(c, "[$]", "").cast(FloatType()),
            "Standard": cast_to(target_type)
        }

        if rule not in handler_map:
            raise ValueError(f"Unsupported rule: {rule}")

        return handler_map[rule]

    # --- Process Each Field ---
    log("Starting conversion process...")

    for dField in aAsset["Fields"]:
        field = dField['Target_Field_Name']
        source_type = dField['Source_Data_Type_Code']
        target_type = dField['Target_Data_Type_Code']
        rule_string = dField.get('Conversion_Rule_String', 'Standard')
        rule_list = [r.strip() for r in rule_string.split(',')]

        log(f"\nField: {field} | Source: {source_type} ➡ Target: {target_type} | Rules: {rule_list}")

        try:
            expr = col(field)
            for r in rule_list:
                handler = get_handler(r, source_type, target_type)
                expr = handler(expr)
                log(f"  ↳ Applied: {r}")
            df = df.withColumn(field, expr)
            log(f"[Success] Field transformed: {field}")
        except Exception as e:
            log(f"[Error] Field: {field} | Reason: {str(e)}")

    log("Conversion process complete.")
    return df


In [5]:
from pyspark.sql.functions import (
    col, upper, lower, to_date, date_format, regexp_replace, udf, substring
)
from pyspark.sql.types import FloatType, IntegerType, StringType, DateType
import datetime
import re

def apply_field_conversions(pDF, aAsset, aLogger=None):
    df = pDF

    def log(msg): aLogger.info(msg) if aLogger else print(msg)

    def cast_to(target_type):
        type_map = {
            "String": StringType(),
            "Int": IntegerType(),
            "Float": FloatType(),
            "Date": DateType()
        }
        return lambda c: c.cast(type_map[target_type])

    def date_to_days_udf():
        def date_to_days(d):
            if d:
                base = datetime.date(1900, 1, 1)
                if isinstance(d, datetime.datetime):
                    d = d.date()
                return (d - base).days if isinstance(d, datetime.date) else None
            return None
        return udf(date_to_days, IntegerType())

    def get_handler(rule, source_type, target_type):
        rule = rule.strip()

        # Left:N substring rule
        if rule.lower().startswith("left:"):
            n = int(rule.split(":")[1])
            return lambda c: substring(c, 1, n)

        # Currency cleanup
        if rule == "???":
            rule = "Currency:Dollar"

        # Date parsing/formatting
        if rule.startswith("Date:"):
            fmt = rule.split("Date:")[1]

            if source_type == "String" and target_type == "Date":
                return lambda c: to_date(c, fmt)
            elif source_type == "Date" and target_type == "String":
                return lambda c: date_format(c, fmt)
            elif source_type == "String" and target_type == "String":
                return lambda c: date_format(to_date(c, 'yyyy-MM-dd'), fmt)
            else:
                raise ValueError(f"Invalid Date conversion: {source_type} ➡ {target_type}")

        # Static conversion rules
        handler_map = {
            "Upper": lambda c: upper(c),
            "Lower": lambda c: lower(c),
            "Decimal:Comma": lambda c: round(regexp_replace(c, ",", ".").cast(FloatType()),2),
            "Currency:Dollar": lambda c: round(regexp_replace(c, "[$]", "").cast(FloatType()),2),
            "DateToInt": lambda c: date_to_days_udf()(c),
            "Standard": cast_to(target_type)
        }

        if rule not in handler_map:
            raise ValueError(f"Unsupported rule: {rule}")

        return handler_map[rule]

    # Apply rules
    log("🔄 Starting field conversion...")

    for dField in aAsset["Fields"]:
        field = dField["Target_Field_Name"]
        source_type = dField["Source_Data_Type_Code"]
        target_type = dField["Target_Data_Type_Code"]
        rule_string = dField.get("Conversion_Rule_String", "Standard")
        rule_list = [r.strip() for r in rule_string.split(",")]

        log(f"\n🛠️  Field: {field} | Source: {source_type} ➡ Target: {target_type} | Rules: {rule_list}")

        try:
            expr = col(field)
            for rule in rule_list:
                handler = get_handler(rule, source_type, target_type)
                expr = handler(expr)
                log(f"   ↪️ Applied: {rule}")
            df = df.withColumn(field, expr)
            log(f"✅ Transformed: {field}")
        except Exception as e:
            log(f"❌ [Error] Field: {field} | Reason: {str(e)}")

    log("✅ Field conversion complete.")
    return df

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, upper, lower, to_date, date_format,
    regexp_replace, udf, substring, round
)
from pyspark.sql.types import FloatType, IntegerType, StringType, DateType, StructType, StructField
from datetime import datetime

# Step 1: Setup Spark
spark = SparkSession.builder.appName("FieldConversionTest").getOrCreate()

schema = StructType([
    StructField("StandardStr", StringType(), True),
    StructField("UpperStr", StringType(), True),
    StructField("LowerStr", StringType(), True),
    StructField("YMDStr", StringType(), True),
    StructField("MDYStr", StringType(), True),
    StructField("IntStr", StringType(), True),
    StructField("FloatStr", StringType(), True),
    StructField("CommaStr", StringType(), True),
    StructField("DollarStr", StringType(), True),
    StructField("IntVal", IntegerType(), True),
    StructField("FloatVal", FloatType(), True),
    StructField("DateCol", DateType(), True),        # True Date value
    StructField("DateTimeStr", StringType(), True)   # DateTime string for Left:10
])


from datetime import date

data = [(
    "ApPLE",               # StandardStr
    "Apple",               # UpperStr
    "Apple",               # LowerStr
    "2025-05-25",          # YMDStr
    "05/25/2025",          # MDYStr
    "123",                 # IntStr
    "123.45",              # FloatStr
    "123,45",              # CommaStr
    "$123.45",             # DollarStr
    2,                     # IntVal
    123.45,                # FloatVal
    date(2025, 5, 25),     # DateCol
    "2025-05-25 12:34:56"  # DateTimeStr
)]

df = spark.createDataFrame(data, schema)
df.show(truncate=False)

aAsset = {
    "Fields": [
        {"Target_Field_Name": "StandardStr",   "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "String", "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "UpperStr",      "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "String", "Conversion_Rule_String": "Upper"},
        {"Target_Field_Name": "LowerStr",      "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "String", "Conversion_Rule_String": "Lower"},
        {"Target_Field_Name": "YMDStr",        "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Date",   "Conversion_Rule_String": "Date:yyyy-MM-dd"},
        {"Target_Field_Name": "MDYStr",        "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Date",   "Conversion_Rule_String": "Date:MM/dd/yyyy"},
        {"Target_Field_Name": "IntStr",        "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Int",    "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "FloatStr",      "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Float",  "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "CommaStr",      "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Float",  "Conversion_Rule_String": "Decimal:Comma"},
        {"Target_Field_Name": "DollarStr",     "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Float",  "Conversion_Rule_String": "???"},
        {"Target_Field_Name": "IntVal",        "Source_Data_Type_Code": "Int",    "Target_Data_Type_Code": "Int",    "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "IntVal",        "Source_Data_Type_Code": "Int",    "Target_Data_Type_Code": "Float",  "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "FloatVal",      "Source_Data_Type_Code": "Float",  "Target_Data_Type_Code": "Float",  "Conversion_Rule_String": "Standard"},
        {"Target_Field_Name": "DateCol",       "Source_Data_Type_Code": "Date",   "Target_Data_Type_Code": "String", "Conversion_Rule_String": "Date:yyyy-MM-dd"},
        # {"Target_Field_Name": "DateCol",       "Source_Data_Type_Code": "Date",   "Target_Data_Type_Code": "String", "Conversion_Rule_String": "Date:MM/dd/yyyy"},
        {"Target_Field_Name": "DateTimeStr",   "Source_Data_Type_Code": "String", "Target_Data_Type_Code": "Date",   "Conversion_Rule_String": "Left:10, Date:yyyy-MM-dd"}
    ]
}


+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-------------------+
|StandardStr|UpperStr|LowerStr|YMDStr    |MDYStr    |IntStr|FloatStr|CommaStr|DollarStr|IntVal|FloatVal|DateCol   |DateTimeStr        |
+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-------------------+
|ApPLE      |Apple   |Apple   |2025-05-25|05/25/2025|123   |123.45  |123,45  |$123.45  |2     |123.45  |2025-05-25|2025-05-25 12:34:56|
+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-------------------+



In [7]:

# Apply conversion
converted_df = apply_field_conversions(pDF=df, aAsset=aAsset)

# Show results
converted_df.show(truncate=False)
converted_df.printSchema()

🔄 Starting field conversion...

🛠️  Field: StandardStr | Source: String ➡ Target: String | Rules: ['Standard']
   ↪️ Applied: Standard
✅ Transformed: StandardStr

🛠️  Field: UpperStr | Source: String ➡ Target: String | Rules: ['Upper']
   ↪️ Applied: Upper
✅ Transformed: UpperStr

🛠️  Field: LowerStr | Source: String ➡ Target: String | Rules: ['Lower']
   ↪️ Applied: Lower
✅ Transformed: LowerStr

🛠️  Field: YMDStr | Source: String ➡ Target: Date | Rules: ['Date:yyyy-MM-dd']
   ↪️ Applied: Date:yyyy-MM-dd
✅ Transformed: YMDStr

🛠️  Field: MDYStr | Source: String ➡ Target: Date | Rules: ['Date:MM/dd/yyyy']
   ↪️ Applied: Date:MM/dd/yyyy
✅ Transformed: MDYStr

🛠️  Field: IntStr | Source: String ➡ Target: Int | Rules: ['Standard']
   ↪️ Applied: Standard
✅ Transformed: IntStr

🛠️  Field: FloatStr | Source: String ➡ Target: Float | Rules: ['Standard']
   ↪️ Applied: Standard
✅ Transformed: FloatStr

🛠️  Field: CommaStr | Source: String ➡ Target: Float | Rules: ['Decimal:Comma']
   ↪️ Appli

In [8]:
converted_df.show(truncate=False)

+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-----------+
|StandardStr|UpperStr|LowerStr|YMDStr    |MDYStr    |IntStr|FloatStr|CommaStr|DollarStr|IntVal|FloatVal|DateCol   |DateTimeStr|
+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-----------+
|ApPLE      |APPLE   |apple   |2025-05-25|2025-05-25|123   |123.45  |123.45  |123.45   |2.0   |123.45  |2025-05-25|2025-05-25 |
+-----------+--------+--------+----------+----------+------+--------+--------+---------+------+--------+----------+-----------+



In [38]:
df.select("DashesDateStr").show(truncate=False)

+-------------+
|DashesDateStr|
+-------------+
|05-25-2025   |
+-------------+



In [1]:
import pandas as pd
from datetime import datetime

# Sample data with actual datetime objects
data = [
    ["Hello World", "hello world", "HELLO WORLD", "LeftFiveTest", "123,45", "$1,234.56", "2024-12-25", "25-12-2024", datetime(2024, 12, 25)],
    ["Example", "example", "EXAMPLE", "ABCDE12345", "456,78", "$987.65", "2025-01-01", "01-01-2025", datetime(2025, 1, 1)],
    ["Test String", "test string", "TEST STRING", "SparkTestVal", "789,01", "$55.40", "2023-06-15", "15-06-2023", datetime(2023, 6, 15)],
    ["Data Here", "data here", "DATA HERE", "Val12345", "321,99", "$10,000.99", "2022-11-11", "11-11-2022", datetime(2022, 11, 11)],
    ["TextLine", "textline", "TEXTLINE", "AbcdeXyz", "98,76", "$65.00", "2020-02-29", "29-02-2020", datetime(2020, 2, 29)],
    ["Entry", "entry", "ENTRY", "Five5Five", "100,00", "$5.50", "2021-01-01", "01-01-2021", datetime(2021, 1, 1)],
    ["MoreText", "moretext", "MORETEXT", "SubStrTest", "888,88", "$88.88", "2026-08-08", "08-08-2026", datetime(2026, 8, 8)],
    ["Sample", "sample", "SAMPLE", "TestValue", "1,23", "$1.23", "2027-07-07", "07-07-2027", datetime(2027, 7, 7)],
    ["Alpha", "alpha", "ALPHA", "12345abcde", "3,14", "$3.14", "2028-09-09", "09-09-2028", datetime(2028, 9, 9)],
    ["Final Row", "final row", "FINAL ROW", "FinalTest", "0,99", "$0.99", "2029-10-10", "10-10-2029", datetime(2029, 10, 10)],
]

# Column headers
columns = [
    "Standard_String",
    "Upper_String",
    "Lower_String",
    "Left5_String",
    "Decimal_Comma",
    "Currency_Dollar",
    "Date_ISO",
    "Date_DDMMYYYY",
    "DateToInt_Test"  # This will be a true date field
]

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Output file path
output_file = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/data/Test_Field_Conversion.csv"

# Write to CSV
df.to_csv(output_file, index=False)

print(f"✅ CSV file saved as: {output_file}")

✅ CSV file saved as: /Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/data/Test_Field_Conversion.csv


In [51]:
import os
os.listdir("/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/data/")

['.DS_Store',
 'spark_columns',
 'TimeProvince.csv',
 'sparkdf',
 'Region.csv',
 'Case.csv']

ModuleNotFoundError: No module named 'DatabaseConstants'

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("DateConversionTest") \
    .getOrCreate()

# Test Data with String Dates, Actual Dates, and Mixed Inputs
test_data = [
    Row(Date_Actual="2025-05-25", Date_ISO="2025-05-25", Date_MDY="05/25/2025"),
    Row(Date_Actual="2024-12-31", Date_ISO="2024-12-31", Date_MDY="12/31/2024")
]

# Create DataFrame
df = spark.createDataFrame(test_data)
df.show(truncate=False)

# Asset configuration with Conversion Rules
aAsset = {
    "Fields": [
        # String to Date Conversion
        {
            "Target_Field_Name": "Date_ISO",
            "Source_Data_Type_Code": "String",
            "Target_Data_Type_Code": "Date",
            "Conversion_Rule_String": "Date:YYYY-MM-DD"
        },
        {
            "Target_Field_Name": "Date_MDY",
            "Source_Data_Type_Code": "String",
            "Target_Data_Type_Code": "Date",
            "Conversion_Rule_String": "Date:MM/DD/YYYY"
        },

        # Date to String Conversion
        {
            "Target_Field_Name": "Date_Actual",
            "Source_Data_Type_Code": "Date",
            "Target_Data_Type_Code": "String",
            "Conversion_Rule_String": "Date:YYYY-MM-DD"
        },
        {
            "Target_Field_Name": "Date_Actual",
            "Source_Data_Type_Code": "Date",
            "Target_Data_Type_Code": "String",
            "Conversion_Rule_String": "Date:MM/DD/YYYY"
        }
    ]
}

# Apply Conversion
converted_df = applyFieldConversions(df, aAsset)

# Show Result
converted_df.show(truncate=False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 11:08:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----------+----------+----------+
|Date_Actual|Date_ISO  |Date_MDY  |
+-----------+----------+----------+
|2025-05-25 |2025-05-25|05/25/2025|
|2024-12-31 |2024-12-31|12/31/2024|
+-----------+----------+----------+



NameError: name 'DIFProcessedTableHelper' is not defined